<p><img src="imagenes/cabecera.png" width="900" align="center"></p>

# Trabajo práctico 2: Modelos de $n$-gramas

## Curso Procesamiento de Lenguaje Natural 

### Maestría en Tecnologías de la información



**Trabajo práctico porpuesto por:** Julio Waissman Vilanova (julio.waissman@unison.mx)

**Desarrollado por:** _Poner tu nombre y correo electrónico aquí_


Este trabajo práctico tiene como objetivo la generalización de los modelos probabilisticos de trigramas que se desarrollaron en el curso a $n$-gramas, con $n \ge 3$. Para modelos de mayor tamaño es necesario contar con textos de mayor amplitud, ya que si nos quedamos con un corpus pequeño, será difícil encontrar 5-gramas, por ejemplo, que se repitan suficientemente para tener estadísticas suficientes para un modelo. Es por esta razón que vamos a utilizar como corpus el libro de *El ingenioso Hidalgo, Don Quijote de la Mancha*.

## 1. Normalización del corpus

Para este corpus vamos a separar el corpus en párrafos (y no en frases como en el ejemplo de las libretas), y los párrafos en tokens, para entrenar por frases. Vamos a considerar que un párrafo se distingue de otro parque hay al menos un linea en blanco entre ambos.

Una vez separado el texto en párrafos, es necesario normalizar la información. Vamos a probar tres variantes:

1. Dejar el texto como está y separar las palabras de los signos de ortografía como tokens diferentes, esperando que el modelo sea capaz de representar correctamente el uso de símbolos ortográficos y el uso de mayúsculas y minúsculas.

2. Eliminar los signos ortográficos, y usar todo el texto en minúsculas (y por lo tanto separa por palabras).

3. Eliminar los signos ortográficos, usar todo el texto en minúsculas, eliminar las palabras de paro, y aplicar un método de *stemming*. El texto no será tan legible pero suponemos que generalizara mejor (vamos a ver).

** Completa las dos funciones de tokenización de párrafo a tokens individuales que faltan.**

In [ ]:
import re
import math
import random
import nltk
import collections



def texto_a_parrafo(texto):
    """
    Generador que separa un texto en párrafos, donde un párrafo
    se distingue de otro por tener al menos una linea en blanco entre ellos.
    
    :param texto: Cadena de caracteres con e texto en cuestion
    
    :yield párrfo por párrafo de texto
    
    """
    parrafo = ''
    for linea in texto.split('\n'):   # Por cada linea de el texto
        parrafo += linea
        if len(linea) == 0 and parrafo is not '':
            yield parrafo
            parrafo = ''

def parrafo_a_tokens_1(parrafo):
    """
    Generador que separa un texto en tokens, los cuales se separan
    por palabras diferentes y tambien por puntuación. El texto no se trata
    y se envía con todo y simbolos, mayúsculas y minusculas.
    
    :param texto: Cadena de caracteres de un párrafo
    
    :yield token por token
    
    """
    return nltk.tokenize.wordpunct_tokenize(parrafo)

def parrafo_a_tokens_2(parrafo):
    """
    Generador que separa un texto en tokens. El texto primero se
    normaliza en minúsculas y se eliminan los signos de puntuación.
    Cada token es una palabra diferente
    
    :param texto: Cadena de caracteres de un párrafo
    
    :yield token por token
    
    """
    # AGREGAR AQUI TU CÖDIGO
    raise NotImplementedError("Falta implementar la función")
    
def parrafo_a_tokens_3(parrafo):
    """
    Generador que separa un texto en tokens. El texto primero se
    normaliza en minúsculas y se eliminan los signos de puntuación.
    Adicionalmente, se eliminan las palabras de paro y se aplica
    un proceso de stemming.
    
    :param texto: Cadena de caracteres de un párrafo
    
    :yield token por token
    
    """
    # AGREGAR AQUI TU CÖDIGO
    raise NotImplementedError("Falta implementar la función")


Y ahora vamos a probar nuestras funciones con el texto seleccionado. Revisa que efectvamente el corpus se separa en párrafos y sean los párrafos correctos del texto. Tambien verifica que no se pase ningun párrafo vacio o sin palabras.

In [ ]:
archivo = "datos/quijote.txt"
with open(archivo, 'r', encoding='utf8') as fp:
    corpus = fp.read()

for (i, parrafo) in enumerate(texto_a_parrafo(corpus)):
    print("Parrafo {}:\n\n{}\n\n".format(i, parrafo))
    if i == 50:
        break

Y ahora revisa que cada uno de los métodos de tokenización de los párrafos realice la operación esperada

In [ ]:
for (i, parrafo) in enumerate(texto_a_parrafo(corpus)):
    tokenizado = ' | '.join(parrafo_a_tokens_1(parrafo))
    print("Parrafo {}:\n\n{}\n\n".format(i, tokenizado))
    if i == 50:
        break

In [ ]:
for (i, parrafo) in enumerate(texto_a_parrafo(corpus)):
    tokenizado = ' | '.join(parrafo_a_tokens_2(parrafo))
    print("Parrafo {}:\n\n{}\n\n".format(i, tokenizado))
    if i == 50:
        break

In [ ]:
for (i, parrafo) in enumerate(texto_a_parrafo(corpus)):
    tokenizado = ' | '.join(parrafo_a_tokens_3(parrafo))
    print("Parrafo {}:\n\n{}\n\n".format(i, tokenizado))
    if i == 50:
        break

## 2. Construyendo un modelo de $n$-gramas

Ahora vamos a contruir un modelo de n-grama de manera similar a como se construyó el modelo de trigramas. Para este ejercicio nos vamos a concentrar en las ideas básicas de los modelos de lenguaje, por lo que no vamos a tomar en cuenta, ni el suavizado, ni las palabras fuera de vocabulario, ya que la complejidad extra en el código no implica una mejor comprensión de las ideas básicas.

Recuerda que en este caso el modelo simplemente es la probabilidad de encontrar un $n$-grama dado, conociendo el $(n-1)$-grama formado por los primeros caracteres de dicho $n$-grama. Para desarrollar esto, vamos a hacer uso de
la función `nltk.ngrams` (además de `collections.Counter` y `collections.defaultdict` por supuesto.

Para este modelo recuerda de insertar el símbolo de inicio `<s>` y el de término `</s>`.

In [ ]:
def modelo_ngrama(corpus, n=3, tokenizador=parrafo_a_tokens_1):
    """
    Genera un modelo linguístico de n-gramas
    
    :param corpus: Una cadena de texto con el corpus completo
    :param n: Int > 2, el grado del n-grama del modelo (default3)
    :tokenizador: Un generador de tokens a partir de un párrafo, defaul parrafo_a_tokens_1
    
    :return: un modelo, el cual es un `defaultdic` tal que modelo[ngrama] es equivalente 
             a la probabilidad P(w_n|w_{n-1}, \ldos, w_1)
    
    """
    n_gramas = (ESCRIBE AQUÍ EL GENERADOR DE N-GRAMAS)
    
    n_menos_uno_gramas = (ESCRIBE AQUÍ EL GENERADOR DE (N-1)-GRAMAS)
    
    n_cuentas = COMPLETA EL CONTADOR DE N-GRAMAS
    
    n_menos_uno_cuentas = ESCRIBE AQUI EL CONTADOR DE (N-1)-GRAMAS
    
    modelo = defaultdict(lambda: 0)
    for ngrama in n_gramas:
        modelo[ngrama] = COMPLETA EL CALCULO DE P(n-grama | (n-1)-grama)
    
    return modelo
    

**Responde a las siguientes preguntas:(en esta misma celda)**

1. ¿Cuales son los 3 5-gramas que más se repiten para el tokenizador `parrafo_a_tokens_1`?
2. ¿Cuales son los 3 5-gramas que más se repiten para el tokenizador `parrafo_a_tokens_2`?
3. ¿Cuales son los 3 5-gramas que más se repiten para el tokenizador `parrafo_a_tokens_3`?
4. ¿Cual es el 4-grama (w1, w2, w3, w4) por el cual el valor de P(w4| w3, w2, w1) es mayor (usando `parrafo_a_tokens_2`)?


In [ ]:
# ANEXA LOS PROGRAMAS NECESARIOS PARA CONTESTAR LAS PREGUNTAS

## 3. Generando párrafos de palabras

Ahora vamos a hacer una función que nos permita generar párrafos a partir de un modelo. La función es muy similar a la que desarrollamos en el curso, con el detalle que esta debe funcionar para $n$-gramas de orden arbitrario.

In [ ]:
def generador_parrafo(modelo):
    """
    genera un parrafo a partir de un modelo de n-grama
    
    :param modelo: Un modelo obtenido a partir de la función modelo_ngrama
    
    :return: Una cadena de caracteres con una sentencia.
    
    """
    n = len(list(modelo.keys())[0])  #  n, tamaño del n-grama
    
    #------------------------------------------------------------------------
    # INSERTAR AQUI EL CÖDIO
    # (revisa las libretas del curso por si puedes reusar código y adaptarlo)
    #------------------------------------------------------------------------
    
    return parrafo


Y ahora lo vamos a probar

In [ ]:
modelo3 = modelo_ngrama(corpus, n=3, tokenizador=parrafo_a_tokens_2)
modelo4 = modelo_ngrama(corpus, n=4, tokenizador=parrafo_a_tokens_2)
modelo5 = modelo_ngrama(corpus, n=5, tokenizador=parrafo_a_tokens_2)
modelo6 = modelo_ngrama(corpus, n=6, tokenizador=parrafo_a_tokens_2)

In [ ]:
parrafo_3 = generador_parrafo(modelo3)
print(20*'=' + '\n' + parrafo_3 + '\n')

parrafo_4 = generador_parrafo(modelo4)
print(20*'=' + '\n' + parrafo_4 + '\n')

parrafo_5 = generador_parrafo(modelo5)
print(20*'=' + '\n' + parrafo_5 + '\n')

parrafo_6 = generador_parrafo(modelo6)
print(20*'=' + '\n' + parrafo_6 + '\n')

Repites varias veces esta última celda y compara los párrafos que encuentras. 

**Describe en esta celda en un párrafo pequeño que es lo que encuentras.**

(_ingresa aquí tu párrafo_)

## 4. Perplejidad

Una cosa es la apreciación subjetiva y otra es el cálculo de la perplejidad de un texto respecto a un modelo dado. Vamos a calcular la perplejidad de un texto respecto a diferentes modelo.

**Desarrolla la función de perplejidad para un párrafo**

In [ ]:
perplejidad(parrafo, modelo):
    """
    Calcula la perplejidad de un modelo frente a un parrafo
    
    :param parrafo: Una cadena de caracteres, la cual vamos a asumir que 
                    es un parrafo (sin saltos de linea ni texto vacio) 
    :param modelo: Un modelo obtenido a partir de la función modelo_ngrama
    
    :return: Un flotante con el valor de la perplejidad (None si la perplejidad es infinita)
    """
    
    #------------------------------------------------------------------------
    # INSERTAR AQUI EL CÖDIO
    # (revisa las libretas del curso por si puedes reusar código y adaptarlo)
    #------------------------------------------------------------------------
    

Y ahora vamos a calcular la perplejidad de `parrafo_6` con los 4 modelos

In [ ]:
print("Para el párrafo {} la perplejidad es:".format(parrafo_6))
print("Para el modelo3: {}".format(perplejidad(parrafo6, modelo3)))
print("Para el modelo4: {}".format(perplejidad(parrafo6, modelo4)))
print("Para el modelo5: {}".format(perplejidad(parrafo6, modelo5)))
print("Para el modelo6: {}".format(perplejidad(parrafo6, modelo6)))

¿Que encontraste? ¿Que conclusiones sacas? ¿Ocurrió algo que no esperabas?

**Escrbe aquí tus conclusiones**

Por útimo vamos a generar un modelo con el corpus completo y vamos a calcular la perplejidad del modelo con el mismo corpus.

In [ ]:
def genera_evalua_modelo(corpus, n=3, tokenizador=parrafo_a_tokens_1):
    """
    Genera y evalúa la perplejidad de un modelo con el mismo corpus con el que se generó
    
    :param corpus: Una cadena de texto con el corpus completo
    :param n: Int > 2, el grado del n-grama del modelo (default3)
    :tokenizador: Un generador de tokens a partir de un párrafo, defaul parrafo_a_tokens_1

    :return la perplejidad promedio de los párrafos
    
    """
    modelo = modelo_ngrama(corpus, n=n, tokenizador=tokenizador)
    perp_acc, m = 0.0, 0
    for parrafo in texto_a_parrafo(corpus):
        perp_acc += perplejidad(parrafo, modelo)
        m += 1
    return perp_acc / m
        

In [ ]:
n = 3
tokenizador = parrafo_a_tokens_1
perp_media = genera_evalua_modelo(corpus, n, tokenizador)
print("Para el modelo con {}-grama, la parplejidad promedio es".format(n, perp_media))

**Completa la tabla siguiente**

| Tokenizador        |   n   | Perplejidad media |
| :----------------  |:-----:| :---------------- | 
| parrafo_a_tokens_1 | 3     |                   |
| parrafo_a_tokens_1 | 4     |                   |
| parrafo_a_tokens_1 | 5     |                   |
| parrafo_a_tokens_2 | 3     |                   |
| parrafo_a_tokens_2 | 4     |                   |
| parrafo_a_tokens_2 | 5     |                   |
| parrafo_a_tokens_3 | 3     |                   |
| parrafo_a_tokens_3 | 4     |                   |
| parrafo_a_tokens_3 | 5     |                   |
